In [5]:
import tensorflow.contrib.layers as tcl
import numpy as np

def leaky_relu(x, leak=0.2):
    f1 = 0.5 * (1 + leak)
    f2 = 0.5 * (1 - leak)
    return f1 * x + f2 * tf.abs(x)


class NeuralNet(object):

    def __init__(self, in_dim, out_dim):
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.hypers = {
            'learning_rate': 0.003, 'hidden_units': 128, 'batch_size': 128,
            'max_iters': 100000, 'eval_iters': 1000}

        self._contruct_model()
        self._init_sess()

    def _contruct_model(self):
        self.input_data = tf.placeholder(
            shape=[None, self.in_dim], dtype=tf.float32)
        self.labels = tf.placeholder(shape=[None, ], dtype=tf.int32)
        self.one_hot_labels = tf.one_hot(
            indices=self.labels, depth=2, on_value=1.0, off_value=0.0)
        h1 = tcl.fully_connected(
            inputs=self.input_data,
            num_outputs=self.hypers['hidden_units'],
            weights_regularizer=tcl.l2_regularizer(0.01),
            activation_fn=leaky_relu)
        h2 = tcl.fully_connected(
            inputs=h1, num_outputs=self.hypers['hidden_units'],
            weights_regularizer=tcl.l2_regularizer(0.01),
            activation_fn=leaky_relu)
        self.logits = tcl.fully_connected(
            inputs=h2, num_outputs=self.out_dim,
            weights_regularizer=tcl.l2_regularizer(0.01),
            activation_fn=tf.nn.sigmoid)
        self.pred = tf.argmax(self.logits, 1)
        self.acc = tf.metrics.accuracy(self.labels, self.pred)
        self.loss = tf.reduce_mean(tf.losses.sigmoid_cross_entropy(
            multi_class_labels=self.one_hot_labels, logits=self.logits))

        optimizer = tf.train.RMSPropOptimizer(
            self.hypers['learning_rate'], 0.9)
        self.train_op = optimizer.minimize(self.loss)

    def _init_sess(self):
        self.sess = tf.Session()
        init = tf.group(tf.global_variables_initializer(),
                        tf.local_variables_initializer())
        self.sess.run(init)

    def fit(self, x, y):
        running_loss = None

        for i in range(self.hypers['max_iters']):
            idx = np.random.randint(0, len(x), self.hypers['batch_size'])
            batch_x, batch_y = x[idx], y[idx]
            feed_dict = {self.input_data: batch_x, self.labels: batch_y}
            l, _ = self.sess.run([self.loss, self.train_op], feed_dict)
            if running_loss:
                running_loss = 0.99 * running_loss + 0.01 * l
            else:
                running_loss = l
            if i % self.hypers['eval_iters'] == 0:
                print('iter: %d running_loss: %.4f' % (i, running_loss))

    def score(self, x, y):
        feed_dict = {self.input_data: x, self.labels: y}
        acc = self.sess.run(self.acc, feed_dict)[1]
        return acc

ModuleNotFoundError: No module named 'tensorflow.contrib'

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.layers import Dense, LSTM, BatchNormalization
from keras.models import Sequential, load_model

Using TensorFlow backend.


In [9]:
df = pd.read_csv("../data/match_features.csv")

In [10]:
data = df.drop(columns=['Unnamed: 0', 'MP'])

In [11]:
batch_size = 64
epoch = 50

# 20% of the data used for testing
test_size = int(.2 * len(data))

In [15]:
# feed the lstm with all features starting from the open price
# still contains the high
input_features = data.iloc[:, :-1].values
input_data = input_features

# train a numpy array of input vectors
sc = MinMaxScaler(feature_range = (0, 1))

input_data[:,:] = sc.fit_transform(input_features[:,:])

In [17]:
input_data.shape

(1143, 82)

In [36]:
X = input_data
y = data.iloc[:, -1].values

In [37]:
# test data taken from the latest (size - (test_size + lookback)) data points
X_test = X[-test_size:]
y_test = y[-test_size:]

# cut the test data out
X_train = X[:-(test_size)]
y_train = y[:-(test_size)]

# reshape x
X_train = X_train.reshape((1, X_train.shape[0], X_train.shape[1]))
X_test = X_test.reshape((1, X_test.shape[0], X_test.shape[1]))

In [38]:
X_train.shape

(1, 915, 82)

In [35]:
model = Sequential()
model.add(LSTM(units=30, return_sequences= True, input_shape=((X_train.shape[0], X_train.shape[1]))))
model.add(Dense(units=1))
model.add(LSTM(units=30, return_sequences=True))
model.add(LSTM(units=30))
model.add(Dense(units=1))
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, y, epochs=epoch, batch_size=batch_size)
model.save(file)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 915, 30)           13560     
_________________________________________________________________
dense_4 (Dense)              (None, 915, 1)            31        
_________________________________________________________________
lstm_9 (LSTM)                (None, 915, 30)           3840      
_________________________________________________________________
lstm_10 (LSTM)               (None, 30)                7320      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 31        
Total params: 24,782
Trainable params: 24,782
Non-trainable params: 0
_________________________________________________________________


ValueError: Error when checking input: expected lstm_8_input to have 3 dimensions, but got array with shape (1143, 82)